# Census data API

Using Census data from ACS 1-Year Detailed Tables <br />
 https://api.census.gov/data/2016/acs/acs1/variables.html <br />
 https://api.census.gov/data/2016/acs/acs1/examples.html <br />
BY state› congressional district	 <br />
    https://api.census.gov/data/2016/acs/acs1?get=B01001_001E,NAME&for=congressional%20district:*&key=YOUR_KEY_GOES_HERE	
    https://api.census.gov/data/2016/acs/acs1?get=B01001_001E,NAME&for=congressional%20district:01&in=state:01&key=YOUR_KEY_GOES_HERE

            

## Variable reference
### Population
B01001_001E: UNWEIGHTED SAMPLE COUNT OF THE POPULATION <br />
B01002_001E - B01002_003E:	MEDIAN AGE <br />
### Education
B15003_001E - B15003_025E: EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER <br />
### Poverty and Income
B17001_001E - B17001_003E, B17001_017E: POVERTY STATUS IN THE PAST 12 MONTHS BY SEX (income below poverty level) <br />
B17001_031E, B17001_032E, B17001_046E: POVERTY STATUS IN THE PAST 12 MONTHS BY SEX (income above poverty level) <br />
B19051_001E - B190051_003E: EARNINGS IN THE PAST 12 MONTHS FOR HOUSEHOLDS <br />
B19052_001E, B19052_002E, B19052_002E: WAGE OR SALARY INCOME IN THE PAST 12 MONTHS FOR HOUSEHOLDS <br />
### Health coverage
B27001_001E, B27001_002E, B27001_030E: HEALTH INSURANCE COVERAGE STATUS BY SEX BY AGE <br />

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import time
from datetime import datetime
from api_keys import api_key

In [3]:
base_url="https://api.census.gov/data/2016/acs/acs1?get="
variables=['B01001_001E']
district_base="&for=congressional%20district:"
district='*'
#base_state='&in=state:'
#state='06'
key='&key='+api_key
url=base_url+variables[0]+district_base+district+key
#print(url)

In [4]:
def getAPIData(purl):
    response = requests.get(purl)
    res=response.json()
    pdf=pd.DataFrame(res)
    return pdf

In [5]:
# set column values as should be instead of being in the first row, drop first row
cdist_df=getAPIData(url)
cdist_df.columns=['population','state','congressional district']
population_df=cdist_df.drop(cdist_df.index[0])
population_df.head()

,population,state,congressional district
1,704457,01,01
2,681621,01,02
3,709482,01,03
4,683273,01,04
5,712529,01,05


In [6]:
#Median Age by congressional district
variables=['B01002_001E,B01002_002E,B01002_003E']
url=base_url
for v in variables:
    url+=v
url+=district_base+district+key
mage_df=getAPIData(url)
mage_df.columns=['median age total','median age male', 'median age female','state','congressional district']
mage_df=mage_df.drop(mage_df.index[0])
mage_df.head()



,median age total,median age male,median age female,state,congressional district
1,39.6,38.3,41.0,01,01
2,38.2,36.7,40.1,01,02
3,38.3,36.6,39.8,01,03
4,40.7,39.7,42.0,01,04
5,40.1,38.4,41.8,01,05


In [7]:
# merge and save as csv
popandmage_df=pd.merge(population_df,mage_df, how='inner', on=['state','congressional district'] )
popandmage_df.head()

,population,state,congressional district,median age total,median age male,median age female
0,704457,01,01,39.6,38.3,41.0
1,681621,01,02,38.2,36.7,40.1
2,709482,01,03,38.3,36.6,39.8
3,683273,01,04,40.7,39.7,42.0
4,712529,01,05,40.1,38.4,41.8


In [8]:
outfile='pop_age_byCDistrict.csv'
popandmage_df.to_csv(outfile)

In [9]:
#Education , age 25 and older by congressional district
variables=[]
for num in range(1,26):
    if num < 10:
        variables.append('B15003_00'+str(num)+'E,')
    else:
        variables.append('B15003_0'+str(num)+'E,')

 

url=base_url
for v in variables:
    url+=v
# remove last comma
url=url[:-1]
url+=district_base+district+key
#print(url)
edu_df=getAPIData(url)

In [10]:
edu_df.head()

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,B15003_001E,B15003_002E,B15003_003E,B15003_004E,B15003_005E,B15003_006E,B15003_007E,B15003_008E,B15003_009E,B15003_010E,...,B15003_018E,B15003_019E,B15003_020E,B15003_021E,B15003_022E,B15003_023E,B15003_024E,B15003_025E,state,congressional district
1,483628,6173,0,89,0,177,147,168,659,1664,...,23782,28984,72702,45045,73720,31803,7552,3179,01,01
2,464210,5620,0,0,372,319,1258,852,1492,2796,...,27056,29275,75042,35913,64329,27290,5782,3768,01,02
3,472077,7650,0,0,30,50,524,557,1471,2651,...,28007,30704,76546,36027,64031,32061,5894,5499,01,03
4,472340,11335,137,31,180,372,1388,1369,1271,5880,...,31974,33496,69006,39050,51327,21812,4541,2741,01,04
